In [ ]:
import numpy as np
import cv2
import time

# define a video capture object 
vid = cv2.VideoCapture(0) 
time.sleep(3)

#high speed, low accuraccy 
#tracker = cv2.TrackerMOSSE_create()

#bit slower, more accurate
tracker = cv2.TrackerCSRT_create()

flag = 1

#starts loop
ret, frame = vid.read() 

bbox = cv2.selectROI("Tracking",frame, False)
tracker.init(frame,bbox)

def drawBox(frame,bbox):
    global x_prev, y_prev
    global flag
    x_temp = 0
    y_temp = 0
    
    x, y, w, h = int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3]) 
    cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (255,0,255), 3,1)
    cv2.putText(frame,"Tracking",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0),2)

     #first frame, no movement
    if flag == 1:
        x_temp = x
        y_temp = y
        flag = 0
    else:
         #save value of x and y prev
        x_temp = x_prev
        y_temp = y_prev
    
    x_prev = x
    y_prev = y
    
    return x,x_temp,y,y_temp

#initialize synth stream 

def update_amp(y_mov,y_prv):
    if(y_mov < 0):
        y_mov = 0
    if(y_prv < 0):
        y_prv = 0
    print("previous y: "+ str(y_prv))
    print("current y: "+ str(y_mov))
    
    #minimum movment to change parameter
    if(abs(y_prv-y_mov)>= 5):
        print("changing amplitude from "+str(y_prv)+" to"+str(y_mov))
        y_mov = y_mov/100
        return y_mov

def update_freq(x_mov,x_prv):
    print("previous x: "+ str(x_prv))
    print("current x: "+ str(x_mov))
    #minimum movment to change parameter
    if(abs(x_prv-x_mov)>= 5):
        print("changing frequency from "+str(x_prv)+" to"+str(x_mov))
        return x_mov
        
while(True): 
    
    timer = cv2.getTickCount() 
    # Capture the video frame 
    ret, frame = vid.read() 
    
    #searches for object in frame
    ret, bbox = tracker.update(frame)
    
    #if object is found in frame
    if ret:
        x,x_prv, y, y_prv = drawBox(frame,bbox)
        
        #update parameter values 
        amp = update_amp(y,y_prv)
        freq = update_freq(x,x_prv)
        
        #update synth stream 
        
        
        
    
    #if object is not found in frame        
    else:
        cv2.putText(frame,"Lost",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)
    
    fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer)
    cv2.putText(frame,str(int(fps)),(75,50), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)
    
    # Display the resulting frame 
    cv2.imshow('frame', frame) 
    
    # press q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 


In [ ]:
def drawBox(frame,bbox):
    global flag 
    global x_prev, y_prev
    temp,temp2 = 0
    
    x, y, w, h = int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3]) 
    cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (255,0,255), 3,1)
    cv2.putText(frame,"Tracking",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0),2)

    #first frame, no movement
    if flag == 1:
        x_mov = 0
        y_mov = 0
        flag = 0
    else:
        x_mov = x-x_prev
        y_mov = y-y_prev
    
    #save previous values 
    x_prev = temp
    y_prev = temp2
    
    x_prev = x
    y_prev = y
    
    return x_mov, y_mov, temp, temp2

